In [240]:
import sys
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.learning_curve import validation_curve
import statsmodels.api as sm
import statsmodels.formula.api as sm
from surprise import Reader, Dataset
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from scipy.special import boxcox1p
import statsmodels.api as sm
import math
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error
import sklearn

In [746]:
def call_api(username, scope):
    token = util.prompt_for_user_token(username,scope,client_id='4849430a2b114eae9d8eaf2c1f14f612',client_secret='11b854b009344409861112df56d9cf44',redirect_uri='http://127.0.0.1:8050/dashboard') 
    return token

In [747]:
token = call_api('mrethana', 'user-top-read')
sp = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=4849430a2b114eae9d8eaf2c1f14f612&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8050%2Fdashboard&scope=user-top-read in your browser


Enter the URL you were redirected to: http://127.0.0.1:8050/dashboard?code=AQA7LVe0B2teY-s9gpABdJNJew6c6j_uzH7aKYI9HxXPhJxgIMfL7_6CC2iZWqXjTWZeiwe2qRDLsvxXi8MBoldEq86IykvYB5zmkgMBYiKXLmTl_UVBFSZlUplR_0dCSnugxybWEbY6rmMfceECK0aYdSIkGdNK4MYcfiWUx2puXwmZplLAvnFX9iqt1GNHwU_-n-MmAaamkXwTohJ1mYOynhk




In [5]:
def create_top_tracks_dict(period):
    list_tracks =[]
    for offset in range(0,500,49):
        x = sp.current_user_top_tracks(49,offset,period)
        temp = [track for track in x['items']]
        list_tracks.extend(temp)
#     import pdb; pdb.set_trace()
    top_track_dict = [{'album':item['album']['name'], 'album_id':item['album']['id'], 'album_release':item['album']['release_date'],'artist':item['artists'][0]['name'],'track_name':item['name'],'track_id':item['id']} for item in list_tracks]
    for item in list_tracks:
        for dic in top_track_dict:
            if item['id'] == dic['track_id']:
                if len(item['artists']) > 1:
                    dic['feature'] = item['artists'][1]['name']
                else:
                    dic['feature'] = 'No Feature'
    return top_track_dict
# lt = create_top_tracks_dict('long_term')





In [79]:
def all_features(dictionary):
    all_features = []
    for item in dictionary:
        all_features.extend(sp.audio_features(item['track_id']))
    return all_features


def add_features(all_features, dictionary):
    for dic in dictionary:
        for item in all_features:
            if dic['track_id'] == item['id']:
                dic['danceability'] = item['danceability']
                dic['energy'] = item['energy']
                dic['loudness'] = item['loudness']
                dic['key'] = item['key']
                dic['acousticness'] = item['acousticness']
                dic['valence'] = item['valence']
                dic['tempo'] = item['tempo']



In [27]:
def regression_evaluation(target, X_list):
    split = train_test_split(X_list, target,random_state=69)
    X_train, X_test, y_train, y_test = split[0], split[1], split[2],split[3]
    X = X_train
    target = y_train
     
    X2 = sm.add_constant(X)
    est = sm.OLS(target, X2)
    est2 = est.fit()
    
    y_predicted = est2.fittedvalues
    y_actual = target
    rmse = math.sqrt(mean_squared_error(y_actual, y_predicted))
    residuals = y_actual-y_predicted
    
    plt.scatter(y_predicted,residuals)
    print(est2.summary())
    plt.scatter(y_predicted,residuals)
    plt.show()
    sns.distplot(residuals)
    print('RMSE=' + str(rmse))

In [38]:
def regression(target, X_list):
    split = train_test_split(X_list, target,random_state=69)
    X_train, X_test, y_train, y_test = split[0], split[1], split[2],split[3]
    X = X_train
    target = y_train
     
    X2 = sm.add_constant(X)
    est = sm.OLS(target, X2)
    est2 = est.fit()
    return est2.rsquared

In [18]:
def poly_regression(x, y, degree_range, xlim_min, xlim_max, ylim_min, ylim_max, alpha = .75,s = 5,width = 2.5):
    plt.figure(figsize = (10,10))
    plt.ylim(ylim_min, ylim_max)
    plt.xlim(xlim_min,xlim_max)
    
    #split inputs into training and test sets
    split = train_test_split(x,y)
    X_train, X_test, y_train, y_test = split[0], split[1], split[2],split[3]
    x = X_train
    y = y_train
    plt.scatter(x, y, s = s, alpha = alpha)
    
        
    for degree in degree_range:
        poly_model = make_pipeline(PolynomialFeatures(degree),
                               LinearRegression())
        poly_model.fit(x[:, np.newaxis], y)
        xfit = np.linspace(xlim_min,xlim_max, 10000)
        yfit = poly_model.predict(xfit[:, np.newaxis])
        plt.plot(xfit, yfit, label = 'degree={}'.format(degree),linewidth=width)
    plt.legend(loc = 'best')


In [67]:
def artist_features(features_for_tracks, dictionary):
    for dic in dictionary:
        for item in features_for_tracks:
            if item == None:
                pass
            else:
                if dic['track_id'] == item['id']:
                    dic['danceability'] = item['danceability']
                    dic['energy'] = item['energy']
                    dic['loudness'] = item['loudness']
                    dic['key'] = item['key']
                    dic['acousticness'] = item['acousticness']
                    dic['valence'] = item['valence']
                    dic['tempo'] = item['tempo']


In [748]:
def artist_tracks_dict(artist_name):
    list_tracks =[]
    for offset in range(0,500,50):
        x = sp.search(artist_name, 50, offset, type = 'track')
        temp = [track for track in x['tracks']['items']]
        list_tracks.extend(temp)
    top_track_dict = [{'album':item['album']['name'], 'album_id':item['album']['id'], 'album_release':item['album']['release_date'],'artist':item['artists'][0]['name'],'track_name':item['name'],'track_id':item['id']} for item in list_tracks]
    for piece in list_tracks:
        for section in top_track_dict:
            if piece['id'] == section['track_id']:
                if len(piece['artists']) > 1:
                    section['feature'] = piece['artists'][1]['name']
                else:
                    section['feature'] = 'No Feature'
    example = top_track_dict
    feat = all_features(example)
    artist_features(feat,example)
    return top_track_dict

In [833]:
token = call_api('mrethana', 'user-top-read')
sp = spotipy.Spotify(auth=token)

In [834]:
artist = artist_tracks_dict('led zeppelin')

retrying ...1secs
retrying ...1secs


In [835]:
a_df = pd.DataFrame(artist)
a_df = a_df.query("artist == 'Led Zeppelin'")

In [836]:
a_df.to_csv('artist.csv')

In [885]:
ldf = pd.read_csv('artist.csv', index_col = 0)

In [886]:
df = pd.read_csv('mrethana.csv', index_col = 0)

In [887]:
def relevant_features(dataframe):
    all_features = ['energy', 'valence', 'tempo','danceability','acousticness','key']
    rs = []
    for feature in all_features:
        yt,max_lambda =boxcox(dataframe.percentile +.01)
        xt,max_lambda=boxcox(dataframe[feature] + .01)
        r = regression(yt,xt)
        rs.append((r,feature))
    return [tuple[1] for tuple in sorted(rs, reverse = True)[0:3]]

In [888]:
from sklearn.metrics.pairwise import cosine_similarity

In [889]:
features = relevant_features(df)

In [890]:
features

['valence', 'energy', 'key']

In [891]:
ldf.key = ldf.key/11
df.key = df.key/11

In [892]:
profile = np.array([df[feat].mean() for feat in features])

In [893]:
profile2d = np.array([df[feat].mean() for feat in features[0:2]])

In [894]:
compare = ldf[[features[0],features[1],features[2]]].values
compare2d = ldf[[features[0],features[1]]].values

In [895]:
test = profile.reshape(1,-1)
test2 = compare[2].reshape(1,-1)

In [896]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

In [897]:
def all_similarities(user,artist):
    cos_list = []
    for item in artist:
        x = cos_sim(user,item)
        cos_list.append(x)
    ldf['cos_similarity'] = cos_list

In [898]:
all_similarities(profile,compare)

In [899]:
profile

array([0.47513946, 0.60469728, 0.47897341])

In [900]:
ldf[(ldf.cos_similarity == ldf.cos_similarity.max())]

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity
59,0.177,The Complete BBC Sessions,6VH2op0GKIl3WNTbZmmcmI,2016-09-16,Led Zeppelin,0.476,0.78,No Feature,0.636364,-9.009,91.609,5cWIRgigrzjTjDBNXQxhAR,Travelling Riverside Blues - 29/6/69 Top Gear,0.634,0.999875


In [901]:
def euclidean_distance(user, artist):   
    return np.sqrt(np.sum((user - artist) ** 2))

In [902]:
ldf.nlargest(5,'cos_similarity')

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity
59,0.177000,The Complete BBC Sessions,6VH2op0GKIl3WNTbZmmcmI,2016-09-16,Led Zeppelin,0.476,0.780,No Feature,0.636364,-9.009,91.609,5cWIRgigrzjTjDBNXQxhAR,Travelling Riverside Blues - 29/6/69 Top Gear,0.634,0.999875
188,0.074500,Coda (Deluxe Edition),56G9UnPmRifbubzPDJfAyd,1982-11-19,Led Zeppelin,0.467,0.835,No Feature,0.636364,-7.084,91.603,3dCarMp6T3GsKCspJgeD0a,Travelling Riverside Blues (BBC Session),0.639,0.999859
283,0.655000,Physical Graffiti (Deluxe Edition),26tH0kjUhkxBEd3ipGkx3Y,1975-02-24,Led Zeppelin,0.759,0.815,No Feature,0.636364,-9.929,127.569,6IBn2H6Axqg4mc4PZW6jRz,Black Country Woman,0.599,0.999618
402,0.000336,Coda (Deluxe Edition),56G9UnPmRifbubzPDJfAyd,1982-11-19,Led Zeppelin,0.297,0.968,No Feature,0.818182,-8.765,175.954,0nBxdeNF1TSfy4OATiR9NK,Walter's Walk (Rough Mix),0.801,0.999569
167,0.586000,Physical Graffiti,1lZahjeu4AhPkg9JARZr5F,1975-02-24,Led Zeppelin,0.755,0.746,No Feature,0.636364,-12.149,127.677,7kMMTfdIkDJpmrkxBlVwEf,Black Country Woman,0.614,0.999496


In [903]:
def all_distance(user,artist):
    e_dist = []
    for item in artist:
        x = euclidean_distance(user,item)
        e_dist.append(x)
    ldf['euclidean'] = e_dist

In [904]:
all_distance(profile,compare)

In [905]:
ldf[(ldf.euclidean == ldf.euclidean.min())]

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity,euclidean
192,0.0637,The Complete BBC Sessions,6VH2op0GKIl3WNTbZmmcmI,2016-09-16,Led Zeppelin,0.367,0.579,No Feature,0.454545,-10.782,128.277,3ztOzed3fKKnQhjlSlt8zC,Since I've Been Loving You - 1/4/71 Paris Theatre,0.369,0.996178,0.111905


In [906]:
ldf = ldf.drop_duplicates('track_name')
rec = ldf.nsmallest(15,'euclidean')

In [907]:
rec

,acousticness,album,album_id,album_release,artist,danceability,energy,feature,key,loudness,tempo,track_id,track_name,valence,cos_similarity,euclidean
192,0.063700,The Complete BBC Sessions,6VH2op0GKIl3WNTbZmmcmI,2016-09-16,Led Zeppelin,0.367,0.579,No Feature,0.454545,-10.782,128.277,3ztOzed3fKKnQhjlSlt8zC,Since I've Been Loving You - 1/4/71 Paris Theatre,0.369,0.996178,0.111905
34,0.656000,Led Zeppelin IV (Remastered Version),5EyIDBAqhnlkAHqvPRwdbX,1971-11-08,Led Zeppelin,0.420,0.662,No Feature,0.363636,-9.828,147.949,6KCjY5kHvgWaWcAV6BBzxO,The Battle Of Evermore,0.378,0.985459,0.161314
68,0.013300,Physical Graffiti,1lZahjeu4AhPkg9JARZr5F,1975-02-24,Led Zeppelin,0.367,0.752,No Feature,0.545455,-8.134,205.470,4hLQfgtzEyxme6w5GiwAEy,The Rover,0.401,0.988867,0.177805
222,0.006390,How The West Was Won (Live [Remastered]),3otvl9PN3kOgk5uwAh1CBL,2003-05-27,Led Zeppelin,0.370,0.677,No Feature,0.454545,-8.552,115.215,08ucu2UJHoZ05NDoBuV0vO,Since I've Been Loving You - Live [Remastered],0.314,0.980576,0.178298
253,0.836000,Led Zeppelin III (Remastered Deluxe Edition),4xGEiQ7La4japmGrREeLlw,1970-10-05,Led Zeppelin,0.350,0.541,No Feature,0.636364,-11.856,85.421,6LMQeGaCMAbU7nlQkDdh5n,Friends - Track - No Vocal,0.412,0.980935,0.181151
234,0.001690,Led Zeppelin III (Remastered Deluxe Edition),4xGEiQ7La4japmGrREeLlw,1970-10-05,Led Zeppelin,0.557,0.399,No Feature,0.454545,-9.062,121.369,2ukj6LJD9be1xklQlXmahK,Since I've Been Loving You - Rough Mix,0.384,0.988231,0.226306
133,0.001480,Physical Graffiti,1lZahjeu4AhPkg9JARZr5F,1975-02-24,Led Zeppelin,0.513,0.740,No Feature,0.636364,-9.290,111.097,2256UUbtBrQJSrbzYpDviT,The Wanton Song,0.577,0.999239,0.231201
116,0.376000,Led Zeppelin III (Remastered),6P5QHz4XtxOmS5EuiGIPut,1970,Led Zeppelin,0.399,0.463,No Feature,0.454545,-13.789,115.850,63jSSgwKQtEdOVRuj85GX7,Hats Off To [Roy] Harper,0.278,0.984761,0.244006
228,0.011600,How The West Was Won (Live [Remastered]),3otvl9PN3kOgk5uwAh1CBL,2003-05-27,Led Zeppelin,0.350,0.817,No Feature,0.363636,-9.686,105.593,2u7UUC5j1AStZ72gtxmXca,Heartbreaker - Live [Remastered],0.406,0.967767,0.251307
167,0.586000,Physical Graffiti,1lZahjeu4AhPkg9JARZr5F,1975-02-24,Led Zeppelin,0.755,0.746,No Feature,0.636364,-12.149,127.677,7kMMTfdIkDJpmrkxBlVwEf,Black Country Woman,0.614,0.999496,0.253023


In [908]:
trace1 = go.Scatter3d(
    x=[profile[0]],
    y=[profile[1]],
    z=[profile[2]],
    mode='markers',
    marker=dict(
        size=18,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

trace2 = go.Scatter3d(
    x=ldf[features[0]],
    y=ldf[features[1]],
    z=ldf[features[2]],
    mode='markers',
    marker=dict(
        size=12,
        color=z,                # set color to an array/list of desired values
        colorscale='Earth',   # choose a colorscale
        opacity=0.4
    ),text= ldf['track_name']
)


data = [trace1,trace2]
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title=features[0]),
                    yaxis = dict(
                        title=features[1]),
                    zaxis = dict(
                        title=features[2]),),
                    width=700,
                    margin=dict(
                    r=20, b=10,
                    l=10, t=10)
                  )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='plot from API (17)')


In [909]:
trace1 = go.Scatter(
    x=[profile[0]],
    y=[profile[1]],
#     z=[profile[2]],
    mode='markers',
    marker=dict(
        size=18,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

trace2 = go.Scatter(
    x=ldf[features[0]],
    y=ldf[features[1]],
#     z=ldf[features[2]],
    mode='markers',
    marker= dict(size= 14,
                    line= dict(width=1),
                    color= z,
                    opacity= 0.3
                   ),
        text= ldf['track_name'])



data = [trace1,trace2]
layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        title=features[0]),
                    yaxis = dict(
                        title=features[1]),
                    ),
                    width=700,
                    margin=dict(
                    r=20, b=10,
                    l=10, t=10)
                  )
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='plot from API (17)')



In [910]:
id_add = list(rec['track_id'].to_dict().values())

In [911]:
token2 = call_api('mrethana', 'playlist-modify-public')
sp2 = spotipy.Spotify(auth=token2)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=4849430a2b114eae9d8eaf2c1f14f612&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8050%2Fdashboard&scope=playlist-modify-public in your browser


Enter the URL you were redirected to: http://127.0.0.1:8050/dashboard?code=AQCH2qBdo3lcwZvkxiz3rvqY9FaE-Oi9Cr7tDuP6jG85vy8YOcDz2yNIFsIsTqIk6sBYahwblRWJnYB_7QSHcV9ZO5TXXhmZFZ-4Gd0EZ4DyEgiOWRXeUSjQw5h3uw2ClUoem0Ndgv1G31VRTthwrNENZl-bWCanelwMb0GV--AMN5kt-dZx9Cp7cDg5VnDyqoxEsBzEt7xs2wD9F2Ne_kI7YeLd4Hs1Fz225VM




In [912]:
return_to_dict = rec['artist'].to_dict()
artist_name = list(return_to_dict.values())[0]
artist_name

'Led Zeppelin'

In [913]:
sp2.user_playlist_create('mrethana',artist_name, True)

{'collaborative': False,
 'description': None,
 'external_urls': {'spotify': 'https://open.spotify.com/user/mrethana/playlist/6zyaj5QA3xhCmxcW8KxtrH'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/users/mrethana/playlists/6zyaj5QA3xhCmxcW8KxtrH',
 'id': '6zyaj5QA3xhCmxcW8KxtrH',
 'images': [],
 'name': 'Led Zeppelin',
 'owner': {'display_name': None,
  'external_urls': {'spotify': 'https://open.spotify.com/user/mrethana'},
  'href': 'https://api.spotify.com/v1/users/mrethana',
  'id': 'mrethana',
  'type': 'user',
  'uri': 'spotify:user:mrethana'},
 'primary_color': None,
 'public': True,
 'snapshot_id': 'MSw4ZjMyMTI5MmZmN2MyOTU1NWY1YzljNDA2MGNjM2ZlMGQ3NTc0YzE1',
 'tracks': {'href': 'https://api.spotify.com/v1/users/mrethana/playlists/6zyaj5QA3xhCmxcW8KxtrH/tracks',
  'items': [],
  'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0},
 'type': 'playlist',
 'uri': 'spotify:user:mrethana:playlist:6zyaj5QA3xhCmxcW8KxtrH'}

In [914]:
newest_id = sp2.user_playlists('mrethana')['items'][0]['id']

In [915]:
sp2.user_playlist_add_tracks('mrethana', newest_id, id_add, None)

{'snapshot_id': 'MixjMDI3MTlmMzI5N2I0NGI2YWRiNjJjNGI1ZGEzY2E3ZWJiNTYxNjRi'}